# UAV Propeller Fault Detection System (Parrot Bebop 2)

This project focuses on the development and validation of fault detection and isolation (FDI) methods for the propulsion system of the **Parrot Bebop 2** unmanned aerial vehicle (UAV). The analysis utilizes inertial sensor data (accelerometer and gyroscope) collected during real-world flight experiments.

### Research Problem
The primary objective is to classify the technical state of the propellers based on vibration signals. We analyze various fault scenarios across four rotors (A, B, C, D), distinguishing between nominal states and specific defects such as chipped edges or bent blades.

### Methodology
This notebook compares two signal processing approaches:
1.  **Time Domain Analysis**
2.  **Frequency Domain Analysis**

Experiment tracking and performance visualization are managed via **Weights & Biases (W&B)**.

# 1. Environment Setup & Global Configuration
Installation of necessary libraries (Weights & Biases for experiment tracking) and importing standard data science modules.

In [2]:
!pip install wandb -q
import wandb
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, precision_score, \
                            recall_score, f1_score, roc_curve, auc, ConfusionMatrixDisplay
from sklearn.preprocessing import label_binarize
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import ModelCheckpoint
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

# Login to W&B
#wandb.login()

# 2. Exploratory Data Analysis
Initial inspection of the accelerometer and gyroscope data from the UAV. We examine the column structure.

## 2.1 Time domain

In [ ]:
!unzip -q /content/Normalized_data.zip

In [ ]:
df_time = pd.read_csv('/content/Bebop2_16g_1kdps_normalized_0000.csv')
df_time

In [ ]:
df_time.info()

## 2.2 Frequency Domain

In [ ]:
!unzip -q /content/FFT_data.zip

In [ ]:
df_freq = pd.read_csv('/content/FFT_data/128_Hann_20_52/Bebop2_16g_FFT_ACCEL_128_Hann_20_52_0000.csv', header=None)
df_freq

In [ ]:
df_freq.info()

# 3. Fault Scenario Mapping

The Bebop 2 flight data is labeled using a 4-digit code (e.g., `1022`), defining the state of each propeller (A, B, C, D):
* **0**: Nominal (Functional propeller).
* **1**: Fault Type I (e.g., chipped edge).
* **2**: Fault Type II (e.g., bent tip/severe damage).

Below, we define the mapping of these physical scenarios to model class labels. Depending on the diagnostic granularity required, the problem can be framed as a **5-class problem** (aggregated by the number of faults) or a **20-class problem** (precise fault configuration).

In [ ]:
# Select Experiment Mode
CLASS_MODE = "20class"   # Options: "5class" (aggregated) or "20class" (precise diagnosis)

In [ ]:
# Precise mapping for 20 unique scenarios (Parrot Bebop 2)
scenario_to_class_20 = {
    "0000": 0,  # Nominal state
    "1000": 1, "0100": 2, "0010": 3, "0001": 4,     # Single faults (Type 1)
    "2000": 5, "0200": 6, "0020": 7, "0002": 8,     # Single faults (Type 2)
    "1100": 9, "1020": 10, "1002": 11, "0120": 12, "0102": 13, "0022": 14, # Dual faults
    "1120": 15, "1102": 16, "1022": 17, "0122": 18, # Triple faults
    "1122": 19, # All propellers faulty
}

# Simplified mapping (Number of faulty rotors)
scenario_to_class_5 = {
    "0000": 0,
    "1000": 1, "0100": 1, "0010": 1, "0001": 1,
    "2000": 1, "0200": 1, "0020": 1, "0002": 1,
    "1100": 2, "1020": 2, "1002": 2, "0120": 2, "0102": 2, "0022": 2,
    "1120": 3, "1102": 3, "1022": 3, "0122": 3,
    "1122": 4,
}

if CLASS_MODE == "5class":
    scenario_to_class = scenario_to_class_5
elif CLASS_MODE == "20class":
    scenario_to_class = scenario_to_class_20
else:
    raise ValueError("Invalid CLASS_MODE selected.")

NUM_CLASSES = len(set(scenario_to_class.values()))
print(f"Experiment Mode: {CLASS_MODE} | Total Classes: {NUM_CLASSES}")

# 3. Time Domain Analysis, data preparation

In this section, we process normalized time-series signals from the accelerometers and gyroscopes. Since the data represents a continuous flight stream, we apply a **sliding window** technique to segment the signal into fixed-length samples (e.g., 256 measurement points).



## 3.1 Data Segmentation

In [ ]:
# Path to normalized time-domain data
DATA_DIR_TIME = r"/content/Normalized_data"
SAMPLE_SIZE = 256      # Window length
N_FEATURES = 24        # Input channels (e.g., 3-axis accel + 3-axis gyro per sensor)

In [ ]:
def make_windows_from_df(df: pd.DataFrame, sample_size: int):
    """
    Segments time-series data into non-overlapping windows.
    """
    data = df.values.astype("float32")
    n_total = len(data)
    n_windows = n_total // sample_size
    if n_windows == 0:
        return np.empty((0, sample_size, data.shape[1]), dtype="float32")
    data = data[:n_windows * sample_size]
    windows = data.reshape(n_windows, sample_size, data.shape[1])
    return windows

In [ ]:
# Data pass through window function

X_list = []
y_list = []

pattern = os.path.join(DATA_DIR_TIME, "Bebop2_16g_1kdps_normalized_*.csv")

for path in glob.glob(pattern):
    fname = os.path.basename(path)
    # ostatni fragment po "_" to kod scenariusza, np. "0000"
    scenario = os.path.splitext(fname)[0].split("_")[-1]

    if scenario not in scenario_to_class:
        print(f"Pomijam {fname} – scenariusz {scenario} nie jest w mapowaniu.")
        continue

    label = scenario_to_class[scenario]

    df = pd.read_csv(path)

    # jeżeli w pliku są inne kolumny niż 24 sensory, tu można wybrać tylko potrzebne:
    # df = df[["A_aX","A_aY",...,"D_gZ"]]

    windows = make_windows_from_df(df, SAMPLE_SIZE)
    if windows.shape[0] == 0:
        print(f"Za mało danych w {fname} na choć jedno okno, pomijam.")
        continue

    X_list.append(windows)
    y_list.append(np.full((windows.shape[0],), label, dtype="int32"))

X = np.concatenate(X_list, axis=0)  # (N, SAMPLE_SIZE, 24)
y = np.concatenate(y_list, axis=0)  # (N,)

print("X shape:", X.shape)
print("y shape:", y.shape, "unikalne etykiety:", np.unique(y))

# 4. Frequency Domain Analysis (FFT)

Mechanical faults in rotating components (such as propellers) generate distinct vibration signatures that are often most discernible in the frequency spectrum.

In this experiment, we utilize data pre-processed via **Fast Fourier Transform (FFT)** using a Hann window to mitigate spectral leakage, which is already done in repo. The input features are vectors of spectral coefficients for each sensor axis.

## 4.1 Spectral Data Structure
The FFT files contain metadata within their filenames (window length, window type, frequency range). The following code parses this information and loads the corresponding spectral coefficients.

In [ ]:
# Configuration for FFT Data
FFT_ROOT      = "FFT_data"
FFT_CONFIG    = "256_Hann_40_104"    # Specific window/range configuration
SENSOR_MODE   = "both"               # "accel", "gyro", or "both"
SAMPLING_RATE = 500.0                # Sampling rate for Bebop 2 inertial sensors

# liczba osi na jeden typ czujnika
N_AXES_SINGLE = 12
N_AXES = 12 if SENSOR_MODE in ("accel", "gyro") else 24

fft_dir = os.path.join(FFT_ROOT, FFT_CONFIG)

In [ ]:
def print_fft_info(fft_dir, sampling_rate=500.0):
    folder_name = os.path.basename(os.path.normpath(fft_dir))
    parts = folder_name.split("_")          # np. ["128","Hann","20","52"]
    measuringWindowLength = int(parts[0])   # 128
    rangeStart = int(parts[-2])             # 20
    rangeStop  = int(parts[-1])             # 52

    freq_res = sampling_rate / measuringWindowLength
    f_start  = (rangeStart - 1) * freq_res
    f_stop   = rangeStop * freq_res

    print(f"Folder FFT: {folder_name}")
    print(f"measuringWindowLength = {measuringWindowLength}")
    print(f"Zakres binów: {rangeStart}–{rangeStop}")
    print(f"Rozdzielczość częstotliwości: {freq_res:.3f} Hz")
    print(f"Zakres częstotliwości: {f_start:.1f} Hz – {f_stop:.1f} Hz")

    return measuringWindowLength, rangeStart, rangeStop, freq_res, f_start, f_stop

In [ ]:
measuringWindowLength, rangeStart, rangeStop, freq_res, f_start, f_stop = print_fft_info(fft_dir, SAMPLING_RATE)

In [ ]:
all_files = glob.glob(os.path.join(fft_dir, "*.csv"))

accel_files = {}  # scenario -> ścieżka
gyro_files  = {}

for path in all_files:
    fname = os.path.basename(path)
    parts = fname.split("_")
    # przykład: Bebop2_16g_FFT_ACCEL_128_Hann_20_52_0000.csv
    # indeksy:   0      1   2   3     4    5    6   7   8
    sensor_type = parts[3]              # "ACCEL" albo "GYRO"
    scenario    = os.path.splitext(parts[-1])[0]  # "0000" itd.

    if sensor_type == "ACCEL":
        accel_files[scenario] = path
    elif sensor_type == "GYRO":
        gyro_files[scenario] = path

print("Znaleziono ACCEL dla scenariuszy:", sorted(accel_files.keys()))
print("Znaleziono GYRO  dla scenariuszy:", sorted(gyro_files.keys()))


In [ ]:
def load_fft_file(path, n_axes=N_AXES_SINGLE):
    # jeśli okaże się, że plik ma nagłówek – zmień na header=0
    df = pd.read_csv(path, header=None)
    data = df.values.astype("float32")   # (n_okien, n_features)
    n_features = data.shape[1]

    if n_features % n_axes != 0:
        raise ValueError(f"{os.path.basename(path)}: {n_features} kolumn "
                         f"nie dzieli się przez {n_axes} osi.")

    n_freq_bins = n_features // n_axes
    data_3d = data.reshape(-1, n_freq_bins, n_axes)  # (n_okien, n_freq_bins, n_axes)
    return data_3d, n_freq_bins

In [ ]:
X_list = []
y_list = []
n_freq_bins_global = None

for scenario, label in scenario_to_class.items():
    cur_X = None

    if SENSOR_MODE == "accel":
        path = accel_files.get(scenario)
        if path is None:
            print(f"[ACCEL] brak pliku dla scenariusza {scenario}, pomijam.")
            continue
        accel_data, n_freq_bins = load_fft_file(path)
        cur_X = accel_data   # (n_okien, n_freq_bins, 12)

    elif SENSOR_MODE == "gyro":
        path = gyro_files.get(scenario)
        if path is None:
            print(f"[GYRO] brak pliku dla scenariusza {scenario}, pomijam.")
            continue
        gyro_data, n_freq_bins = load_fft_file(path)
        cur_X = gyro_data    # (n_okien, n_freq_bins, 12)

    elif SENSOR_MODE == "both":
        path_a = accel_files.get(scenario)
        path_g = gyro_files.get(scenario)
        if path_a is None or path_g is None:
            print(f"[BOTH] brak ACCEL lub GYRO dla {scenario}, pomijam.")
            continue

        accel_data, n_freq_bins_a = load_fft_file(path_a, n_axes=N_AXES_SINGLE)
        gyro_data,  n_freq_bins_g = load_fft_file(path_g, n_axes=N_AXES_SINGLE)

        if accel_data.shape[0] != gyro_data.shape[0] or n_freq_bins_a != n_freq_bins_g:
            raise ValueError(f"Niezgodne rozmiary ACCEL/GYRO dla scenariusza {scenario}")

        # sklejanie po osi „kanałów”: 12 (ACCEL) + 12 (GYRO) = 24
        cur_X = np.concatenate([accel_data, gyro_data], axis=-1)  # (..., n_freq_bins, 24)
        n_freq_bins = n_freq_bins_a

    else:
        raise ValueError("SENSOR_MODE musi być 'accel', 'gyro' albo 'both'")

    # ustaw / sprawdź globalną liczbę binów
    if n_freq_bins_global is None:
        n_freq_bins_global = n_freq_bins
    elif n_freq_bins_global != n_freq_bins:
        raise ValueError("Różne n_freq_bins między plikami, coś jest nie tak.")

    X_list.append(cur_X)
    y_list.append(np.full((cur_X.shape[0],), label, dtype="int32"))

# Sklejenie wszystkiego
X = np.concatenate(X_list, axis=0)   # (N, n_freq_bins, N_AXES)
y = np.concatenate(y_list, axis=0)   # (N,)

print("X shape:", X.shape)
print("y shape:", y.shape, "unikalne etykiety:", np.unique(y))

n_freq_bins = n_freq_bins_global
input_shape = (n_freq_bins, N_AXES)
print("input_shape modelu:", input_shape)


# 5. Models Creation

## 5.1 ANN - MLP

In [ ]:
# Initialize W&B Experiment, CONFIG
wandb.init(
    project="UAV-FDI-Bebop2",
    name="Freq_MLP",
    config={
        "domain": "freq",
        "sensor_mode": SENSOR_MODE,
        "input_shape": input_shape,
        "output_shape": NUM_CLASSES,
        "layer_1": 120,
        "dropout_rate": 0.5,
        "l2_value": 0.01,
        "activation": "relu",
        "final_activation": "softmax",
        "optimizer": "adam",
        "loss": "sparse_categorical_crossentropy",
        "metric": "accuracy",
        "learning_rate": 0.001,
        "epochs": 500,
        "batch_size": 1000
    }
)


config = wandb.config

# 6. Data division - train/val/test

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)


train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)) \
    .shuffle(buffer_size=len(X_train)) \
    .batch(config.batch_size) \
    .prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)) \
    .batch(config.batch_size) \
    .prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)) \
    .batch(config.batch_size)

# 6. Training Loop

In [ ]:
# ==============================================================================
# 3. TRENOWANIE
# ==============================================================================

print("Rozpoczynam trenowanie...")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=config.epochs,
    callbacks=[
        # Loguje przebieg treningu (krzywe live)
        WandbMetricsLogger(log_freq=5),
        # Zapisuje najlepszy model (wymagane .keras dla Keras 3)
        ModelCheckpoint(
            filepath="checkpoints/model_best.keras",
            save_best_only=True,
            monitor="val_accuracy",
            mode="max",
        )
    ]
)

# ==============================================================================
# 4. EWALUACJA I OBLICZANIE METRYK
# ==============================================================================

print("\nGenerowanie predykcji na zbiorze testowym...")
y_true_list = []
y_proba_list = []

for x_batch, y_batch in test_ds:
    y_true_list.append(y_batch.numpy())
    y_proba_list.append(model.predict(x_batch, verbose=0))

y_true = np.concatenate(y_true_list, axis=0)   # (N,)
y_proba = np.concatenate(y_proba_list, axis=0) # (N, NUM_CLASSES)
y_pred = np.argmax(y_proba, axis=1)            # (N,)

# Upewniamy się, że y_true to inty
if y_true.ndim > 1:
    y_true_int = np.argmax(y_true, axis=1)
else:
    y_true_int = y_true.astype(int)

# --- Metryki Liczbowe ---
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
prec = precision_score(y_true_int, y_pred, average="macro")
rec = recall_score(y_true_int, y_pred, average="macro")
f1 = f1_score(y_true_int, y_pred, average="macro")

print(f"Loss: {test_loss:.4f}, Acc: {test_acc:.4f}, F1: {f1:.4f}")

# ==============================================================================
# 5. TWORZENIE WYKRESÓW (MATPLOTLIB)
# ==============================================================================

# A. Loss Plot
fig_loss = plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Training Loss')
plt.legend(); plt.grid(True)

# B. Accuracy Plot
fig_acc = plt.figure(figsize=(8, 5))
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Training Accuracy')
plt.legend(); plt.grid(True)

# C. Confusion Matrix
cm = confusion_matrix(y_true_int, y_pred)
fig_cm, ax = plt.subplots(figsize=(8, 8))
ConfusionMatrixDisplay(cm).plot(ax=ax, cmap='Blues', values_format='d')
plt.title('Confusion Matrix')

# D. ROC Curve
y_true_bin = label_binarize(y_true_int, classes=np.arange(NUM_CLASSES))
fpr, tpr, roc_auc = {}, {}, {}
for i in range(NUM_CLASSES):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

fig_roc = plt.figure(figsize=(10, 8))
for i in range(NUM_CLASSES):
    plt.plot(fpr[i], tpr[i], label=f'Cls {i} (AUC={roc_auc[i]:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.title('ROC Curve')
# Legenda poza wykresem, żeby nie zasłaniała przy wielu klasach
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()

# 2. RĘCZNE WYSŁANIE MODELU NA KOŃCU
# Robimy to tylko raz, tuż przed wandb.finish()
print("Wysyłanie najlepszego modelu do W&B...")
wandb.save("checkpoints/model_best.keras")

# ==============================================================================
# 6. LOGOWANIE FINALNE (Podział na Summary i Media)
# ==============================================================================

# 1. LICZBY -> wandb.summary (Tylko do tabeli, BEZ kropek na wykresach)
wandb.summary["test_loss"] = test_loss
wandb.summary["test_accuracy"] = test_acc
wandb.summary["test_precision"] = prec
wandb.summary["test_recall"] = rec
wandb.summary["test_f1_score"] = f1

# 2. OBRAZKI -> wandb.log (Do sekcji Media / Custom Charts)
wandb.log({
    "charts/loss_history": wandb.Image(fig_loss),
    "charts/acc_history": wandb.Image(fig_acc),
    "charts/confusion_matrix": wandb.Image(fig_cm),
    "charts/roc_curve": wandb.Image(fig_roc)
})

# Sprzątanie pamięci RAM (zamykamy wykresy matplotlib)
plt.close('all')

print("Proces zakończony. Wyniki wysłane do W&B.")
wandb.finish()

# 7. Advanced Evaluation and Quality Metrics

# 6. Model Export

To facilitate deployment on the drone's embedded platform or for future inference tasks, we serialize the trained model along with a metadata file. The `metadata.json` file contains critical configuration details regarding sensor setup and class mapping, which are essential for correctly interpreting model predictions in a production environment.

In [ ]:
import os
import json

# katalog na zapisane modele
SAVE_DIR = "saved_models"
os.makedirs(SAVE_DIR, exist_ok=True)

MODEL_PATH    = os.path.join(SAVE_DIR, "uav_fft_classifier.keras")  # format Keras
METADATA_PATH = os.path.join(SAVE_DIR, "metadata.json")

# 1) zapis samego modelu
model.save(MODEL_PATH)
print("Zapisano model do:", MODEL_PATH)

# 2) zapis podstawowych metadanych (przydadzą się przy wczytywaniu)
metadata = {
    "sensor_mode": SENSOR_MODE,             # "accel" / "gyro" / "both"
    "num_classes": int(NUM_CLASSES),
    "n_freq_bins": int(n_freq_bins),
    "n_axes": int(N_AXES),
    "scenario_to_class": scenario_to_class, # mapping klas
}

with open(METADATA_PATH, "w") as f:
    json.dump(metadata, f, indent=2)

print("Zapisano metadane do:", METADATA_PATH)
